**DONT RUN UNLESS ON NEW SERVER** <br>
pip instal for needed libraries

In [10]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Invalid requirement: 'pip install ib_insync pandas numpy matplotlib seaborn statsmodels scipy scikit-learn ta-lib yfinance fredapi dash streamlit arch plotly tensorflow requests beautifulsoup4' (from line 1 of requirements.txt)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ib_insync import ib
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import norm
from ibkr_connection import connect_ibkr, disconnect_ibkr

**CONNECTION TO IBKR**

In [8]:
# Step 1: Connect to IBKR
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)  # Connect to IB Gateway/TWS

NameError: name 'IB' is not defined

**HISTORICAL DATA FROM US TREASURY 10Y YIELD**

In [7]:
# Step 2: Fetch Historical Data (Example: US Treasury Yield 10Y)
contract = Index('IR.10YR', 'EUREX')  # Replace with appropriate IBKR symbol
bars = ib.reqHistoricalData(
    contract,
    endDateTime='',
    durationStr='2 Y',
    barSizeSetting='1 month',
    whatToShow='MIDPOINT',
    useRTH=True,
    formatDate=1
)

# Convert to DataFrame
df = util.df(bars)
df.set_index('date', inplace=True)
df['Interest_Rate'] = df['close']

# Step 3: Visualize Historical Data
plt.figure(figsize=(10, 6))
plt.plot(df['Interest_Rate'], label='10Y Treasury Yield (IBKR)')
plt.title('Historical 10Y Treasury Yield')
plt.legend()
plt.show()

# Step 4: Fit ARIMA Model
model = ARIMA(df['Interest_Rate'], order=(1, 1, 1))
fit = model.fit()

# Step 5: Forecast with Monte Carlo Simulation
n_forecast = 12  # Forecast 12 months ahead
forecast, se, conf_int = fit.forecast(steps=n_forecast, alpha=0.05)

simulations = 1000
simulated_forecasts = np.zeros((simulations, n_forecast))

for i in range(simulations):
    noise = np.random.normal(0, se)
    simulated_forecasts[i] = forecast + noise

mean_forecast = np.mean(simulated_forecasts, axis=0)
percentiles = np.percentile(simulated_forecasts, [5, 25, 50, 75, 95], axis=0)

# Step 6: Plot Probabilistic Forecast
plt.figure(figsize=(10, 6))
plt.plot(df['Interest_Rate'], label='Historical Rates')
plt.plot(pd.date_range(start=df.index[-1], periods=n_forecast+1, freq='M')[1:], mean_forecast, label='Forecast')
plt.fill_between(pd.date_range(start=df.index[-1], periods=n_forecast+1, freq='M')[1:], 
                 percentiles[0], percentiles[-1], color='gray', alpha=0.3, label='90% CI')
plt.title('Probabilistic Forecast for 10Y Treasury Yield (IBKR)')
plt.legend()
plt.show()

# Step 7: Disconnect IBKR
ib.disconnect()


NameError: name 'Index' is not defined

In [ ]:
# Step 3: Visualize Historical Data
plt.figure(figsize=(10, 6))
plt.plot(df['Interest_Rate'], label='10Y Treasury Yield (IBKR)')
plt.title('Historical 10Y Treasury Yield')
plt.legend()
plt.show()

In [ ]:
# Step 4: Fit ARIMA Model
model = ARIMA(df['Interest_Rate'], order=(1, 1, 1))
fit = model.fit()

In [ ]:
# Step 5: Forecast with Monte Carlo Simulation
n_forecast = 12  # Forecast 12 months ahead
forecast, se, conf_int = fit.forecast(steps=n_forecast, alpha=0.05)

simulations = 1000
simulated_forecasts = np.zeros((simulations, n_forecast))

for i in range(simulations):
    noise = np.random.normal(0, se)
    simulated_forecasts[i] = forecast + noise

mean_forecast = np.mean(simulated_forecasts, axis=0)
percentiles = np.percentile(simulated_forecasts, [5, 25, 50, 75, 95], axis=0)


In [ ]:
# Step 6: Plot Probabilistic Forecast
plt.figure(figsize=(10, 6))
plt.plot(df['Interest_Rate'], label='Historical Rates')
plt.plot(pd.date_range(start=df.index[-1], periods=n_forecast+1, freq='M')[1:], mean_forecast, label='Forecast')
plt.fill_between(pd.date_range(start=df.index[-1], periods=n_forecast+1, freq='M')[1:], 
                 percentiles[0], percentiles[-1], color='gray', alpha=0.3, label='90% CI')
plt.title('Probabilistic Forecast for 10Y Treasury Yield (IBKR)')
plt.legend()
plt.show()

# Step 7: Disconnect IBKR
ib.disconnect()